DMN Pipeline:

In [ ]:
import requests

from typing import List

from haystack import Document, Pipeline, component
from haystack import component
from haystack.components.builders import PromptBuilder
from haystack.components.generators.openai import OpenAIGenerator
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument

In [ ]:
import pandas as pd

In [ ]:
@component
class ReviewData:

    @component.output_types(reviews=str, company_name=str)
    def run(self, csv_file_path: str, company: str):
        dataframe1 = pd.read_csv(csv_file_path)
        return {"reviews": dataframe1[["Review Text","Reviewer Name"]].head(10).to_dict("records"), "company_name": company}
        
review_data = ReviewData()

output = review_data.run(csv_file_path="fashionnova_reviews.csv", company="fashionnova")
output["reviews"]

In [ ]:
from haystack.components.generators import HuggingFaceLocalGenerator

generator2 = HuggingFaceLocalGenerator(
    model="google/flan-t5-large",
    task="text2text-generation",
    generation_kwargs={"max_new_tokens": 100000, "temperature": 0.9})

generator2.warm_up()
print(generator2.run("Who is the best American actor?"))

In [ ]:
from haystack_integrations.components.generators.amazon_bedrock import AmazonBedrockGenerator

generator = AmazonBedrockGenerator(
        model="amazon.titan-text-lite-v1",
        max_length=99
)

# print(generator.run("Who is the best American actor?"))

In [ ]:
review_data = ReviewData()

template = """ You will be provided a few of the top reviews for {{ company_name }}. For each review, which reviewers have positive sentiment?
Reviews:  
{% for review in reviews %}
  review:\n
  {{ review['Reviewer Name'] }} wrote {{ review['Review Text'] }}
{% endfor %}  
"""
prompt1 = PromptBuilder(template=template)
print(prompt1.run(template_variables={"reviews": review_data.run(csv_file_path="fashionnova_reviews.csv", company="fashionnova")["reviews"], "company_name": "fashionnova" }))


In [ ]:
review_data = ReviewData()

template = """ You will be provided a few of the top reviews for {{ company_name }}. Based on the Reviews, Which reviewers have negative sentiment?
Reviews:  
{% for review in reviews %}
  review:\n
  {{ review['Reviewer Name'] }} wrote {{ review['Review Text'] }}
{% endfor %}  
"""
prompt1 = PromptBuilder(template=template)
print(prompt1.run(template_variables={"reviews": review_data.run(csv_file_path="fashionnova_reviews.csv", company="fashionnova")["reviews"], "company_name": "fashionnova" }))

# llm = OpenAIGenerator()

dialogue_builder1 = Pipeline()
dialogue_builder1.add_component("review_data", review_data)
dialogue_builder1.add_component("prompt", prompt1)
dialogue_builder1.add_component("llm", generator)

dialogue_builder1.connect("review_data.reviews", "prompt.reviews")
dialogue_builder1.connect("review_data.company_name", "prompt.company_name")
dialogue_builder1.connect("prompt", "llm")

In [ ]:
dialogue_builder1.show()

In [ ]:
dialogue1 = dialogue_builder1.run({"review_data": {"csv_file_path":"fashionnova_reviews.csv", "company":"fashionnova"}})

print(dialogue1["llm"])